In [31]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn import model_selection, preprocessing, ensemble
import xgboost as xgb

In [32]:
train = pd.read_csv("train.csv")


In [33]:

train = train[train.logerror > -0.4 ]
train = train[train.logerror < 0.419 ]
train = train.reset_index(drop=True)

In [34]:
train.shape

(88528, 164)

In [35]:
train_y = train.logerror


In [36]:
train.drop(['propertyzoningdesc', 'logerror', 'nb_count_w'], axis = 1, inplace = True)
#transactiondate

In [37]:
train.dtypes

parcelid                            int64
basementsqft                      float64
bathroomcnt                       float64
bedroomcnt                        float64
buildingqualitytypeid             float64
calculatedbathnbr                 float64
finishedfloor1squarefeet          float64
calculatedfinishedsquarefeet      float64
finishedsquarefeet12              float64
finishedsquarefeet13              float64
finishedsquarefeet15              float64
finishedsquarefeet50              float64
finishedsquarefeet6               float64
fips                              float64
fireplacecnt                      float64
fullbathcnt                       float64
garagecarcnt                      float64
garagetotalsqft                   float64
latitude                          float64
longitude                         float64
lotsizesquarefeet                 float64
poolcnt                           float64
poolsizesum                       float64
pooltypeid10                      

In [38]:
train.head()

,parcelid,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,...,propertycountylandusecode_1722,propertycountylandusecode_200,propertycountylandusecode_34,propertycountylandusecode_38,propertycountylandusecode_6050,propertycountylandusecode_73,propertycountylandusecode_8800,propertycountylandusecode_96,fireplaceflag_True,taxdelinquencyflag_Y
0,11016594,NaN,2.0,3.0,4.0,2.0,NaN,1684.0,1684.0,NaN,...,0,0,0,0,0,0,0,0,0,0
1,14366692,NaN,3.5,4.0,NaN,3.5,NaN,2263.0,2263.0,NaN,...,0,0,0,0,0,0,0,0,0,0
2,12098116,NaN,3.0,2.0,4.0,3.0,NaN,2217.0,2217.0,NaN,...,0,0,0,0,0,0,0,0,0,0
3,12643413,NaN,2.0,2.0,4.0,2.0,NaN,839.0,839.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,14432541,NaN,2.5,4.0,NaN,2.5,NaN,2283.0,2283.0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [39]:
train_y.describe()

count    88528.000000
mean         0.007628
std          0.083459
min         -0.399000
25%         -0.025300
50%          0.005000
75%          0.037300
max          0.418700
Name: logerror, dtype: float64

In [40]:
print(train.shape)
print(train_y.shape)


(88528, 161)
(88528,)


In [41]:
#dtrain = xgb.DMatrix(train, train_y, feature_names=train.columns.values)
import gc
gc.collect()


590

In [42]:
#y_mean = np.mean(train_y)


def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=321, num_rounds=3000):
    param = {}
    param['objective'] = 'reg:linear'
    param['eta'] = 0.1
    param['max_depth'] = 2
    param['silent'] = 1
    param['eval_metric'] = "rmse"
    param['min_child_weight'] = 1
    param['nthread'] = 16 
#    param['base_score'] = y_mean 
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    param['num_rounds'] = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20, verbose_eval = 10)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model



In [43]:
cv_scores = []
kf = model_selection.KFold(n_splits=10, shuffle=True, random_state=2016)
for dev_index, val_index in kf.split(range(train.shape[0])):
        dev_X, val_X = train.loc[dev_index,:], train.loc[val_index,:]
        dev_Y, val_Y = train_y[dev_index], train_y[val_index]
        preds, model = runXGB(dev_X, dev_Y, val_X, val_Y)
        cv_scores.append(mean_absolute_error(val_Y, preds))
        print(cv_scores)
        break



[0]	train-rmse:0.450888	test-rmse:0.451161
Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.

Will train until test-rmse hasn't improved in 20 rounds.
[10]	train-rmse:0.175487	test-rmse:0.175736
[20]	train-rmse:0.098958	test-rmse:0.099121
[30]	train-rmse:0.084934	test-rmse:0.085084
[40]	train-rmse:0.082976	test-rmse:0.083105
[50]	train-rmse:0.082676	test-rmse:0.082809
[60]	train-rmse:0.082578	test-rmse:0.082755
[70]	train-rmse:0.082508	test-rmse:0.082707
[80]	train-rmse:0.082453	test-rmse:0.082686
[90]	train-rmse:0.082389	test-rmse:0.082675
[100]	train-rmse:0.082341	test-rmse:0.082675
[110]	train-rmse:0.082288	test-rmse:0.082642
[120]	train-rmse:0.082235	test-rmse:0.082636
[130]	train-rmse:0.082186	test-rmse:0.08261
[140]	train-rmse:0.082143	test-rmse:0.082597
[150]	train-rmse:0.082106	test-rmse:0.08259
[160]	train-rmse:0.082065	test-rmse:0.082589
Stopping. Best iteration:
[145]	train-rmse:0.082121	test-rmse:0.082586

[0.052751649369538058]


In [13]:
#Stopping. Best iteration:
[154]	train-rmse:0.082119	test-rmse:0.082577

[0.052791524642169206]



Stopping. Best iteration:
[145]	train-rmse:0.082121	test-rmse:0.082586

[0.052751649369538058]

In [45]:

test = pd.read_pickle('test.pkl')


In [46]:

test.drop(['propertyzoningdesc', 'nb_count_w'], axis =1, inplace = True)

#del prop

import gc

gc.collect()

129

In [47]:
preds, model = runXGB(train, train_y, test, num_rounds=145)

#del [train, test, model]


In [48]:
imp_df_xgb = pd.DataFrame()
imp_df_xgb['Predictor'] = model.get_fscore().keys()
imp_df_xgb['Importance'] = model.get_fscore().values()

imp_df_xgb = imp_df_xgb.sort_values(['Importance'], ascending=False)

imp_df_xgb[:20]

,Predictor,Importance
1,taxamount,39
0,finishedsquarefeet12,32
4,structuretaxvaluedollarcnt,29
3,calculatedfinishedsquarefeet,25
11,landtaxvaluedollarcnt,22
12,yearbuilt,21
5,taxvaluedollarcnt,20
19,longitude,16
20,finishedsquarefeet15,15
8,censustractandblock,15


In [49]:

#pred = model.predict(test)
y_pred=[]

for i,predict in enumerate(preds):
    y_pred.append(str(round(predict,4)))
y_pred=np.array(y_pred)

output = pd.DataFrame({'ParcelId': test['parcelid'].astype(np.int32),
        '201610': y_pred, '201611': y_pred, '201612': y_pred,
        '201710': y_pred, '201711': y_pred, '201712': y_pred})
# set col 'ParceID' to first col
cols = output.columns.tolist()
cols = cols[-1:] + cols[:-1]
output = output[cols]
from datetime import datetime
output.to_csv('sub{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False)